In [65]:

%load_ext autoreload
%autoreload 

import numpy as np
import seaborn as sns
import scipy
import skimage
sns.set_style("ticks")
from pathlib import Path

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [66]:
!pip install openpyxl

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
DEPRECATION: torch-tensorrt 1.1.0a0 has a non-standard dependency specifier torch>=1.10.0+cu113<1.11.0. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of torch-tensorrt or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip


# Get the data 

In [67]:
# in this file is the url to the spreadsheet with the data
# PIGLEG_SPREADSHEET_URL=https://docs.google.com/spreadsheets/d/1G55kXxcJ0...piLaK0snddRU-jzU/edit?usp=sharing
env_paths = list(Path(".").glob("../**/.env.dev"))
print(env_paths)
# find line with PIGLEG_SPREADSHEET_URL and give me the value after the "="
PIGLEG_SPREADSHEET_URL = [line.split("=", 1)[1].strip() for line in open(env_paths[0], "r") if line.startswith("PIGLEG_SPREADSHEET_URL")][0]
PIGLEG_SPREADSHEET_URL = PIGLEG_SPREADSHEET_URL.replace("\"", "")
PIGLEG_SPREADSHEET_URL = PIGLEG_SPREADSHEET_URL.replace("edit?usp=sharing", "export?format=csv")
# PIGLEG_SPREADSHEET_URL

[PosixPath('../.env.dev')]


In [68]:
import pandas as pd
url = PIGLEG_SPREADSHEET_URL
df = pd.read_csv(url)

In [69]:
df.tail()

,finished_at,filename,report_url,duration_s_get_parameters_for_crop_rotate_rescale,orig fps,orig totalframecount,collections,orig frame_width,orig frame_height,ffmpeg_subprocess_params,...,Needle holder bbox knot 1 length [m],Needle holder bbox knot 1 visibility [s],Needle holder bbox knot 1 velocity,Needle holder bbox knot 1 velocity std,Needle holder bbox knot 1 velocity above threshold,Needle holder bbox knot 1 velocity threshold [m/s],Needle holder bbox knot 1 unit,Needle holder bbox knot 1 visibility [%],Forceps bbox knot 1 area presence [%],Forceps bbox knot 1 median area presence [%]
293,2024-09-17 15:38,Pre_-Test_Isabel_Linden.mp4,http://suran.kky.zcu.cz:8000//uploader/web_rep...,"11,35641861",25.0,8940.0,"['pre-test post-test', 'dataset 2024-06', 'ski...",1920.0,1020.0,ffmpeg -i /webapps/piglegsurgery/piglegsurgery...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
294,2024-09-17 15:38,Fehringer_Pascal__post_test.mp4,http://suran.kky.zcu.cz:8000//uploader/web_rep...,"22,75856137",25.0,2130.0,"['pre-test post-test', 'dataset 2024-06', 'ski...",1280.0,804.0,ffmpeg -i /webapps/piglegsurgery/piglegsurgery...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
295,2024-09-17 15:44,"Hilger,Anke-Posttest.mp4",http://suran.kky.zcu.cz:8000//uploader/web_rep...,"21,92870426",25.0,7651.0,"['pre-test post-test', 'dataset 2024-06', 'ski...",2560.0,1920.0,ffmpeg -i /webapps/piglegsurgery/piglegsurgery...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
296,2024-09-17 15:45,Schiesser_David_post_test.mp4,http://suran.kky.zcu.cz:8000//uploader/web_rep...,"9,279839754",25.0,7543.0,"['pre-test post-test', 'dataset 2024-06', 'ski...",1280.0,720.0,ffmpeg -i /webapps/piglegsurgery/piglegsurgery...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
297,2024-09-17 15:57,2023-02-21_SchuldtSalome_Einzelknopfnaht_am_Sc...,http://suran.kky.zcu.cz:8000//uploader/web_rep...,"12,82772851",27.0,37679.0,"['Einzelknopfnaht', 'dataset 2024-06', 'Einzel']",1920.0,1080.0,ffmpeg -i /webapps/piglegsurgery/piglegsurgery...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [70]:
df.columns

Index(['finished_at', 'filename', 'report_url',
       'duration_s_get_parameters_for_crop_rotate_rescale', 'orig fps',
       'orig totalframecount', 'collections', 'orig frame_width',
       'orig frame_height', 'ffmpeg_subprocess_params',
       ...
       'Needle holder bbox knot 1 length [m]',
       'Needle holder bbox knot 1 visibility [s]',
       'Needle holder bbox knot 1 velocity',
       'Needle holder bbox knot 1 velocity std',
       'Needle holder bbox knot 1 velocity above threshold',
       'Needle holder bbox knot 1 velocity threshold [m/s]',
       'Needle holder bbox knot 1 unit',
       'Needle holder bbox knot 1 visibility [%]',
       'Forceps bbox knot 1 area presence [%]',
       'Forceps bbox knot 1 median area presence [%]'],
      dtype='object', length=777)

In [71]:

df.columns[df.columns.str.contains("titch 0 ")]


Index(['Stitch 0 duration [s]', 'Stitch 0 start at [s]',
       'Needle holder stitch 0 area presence [%]',
       'Needle holder stitch 0 median area presence [%]',
       'Needle holder stitch 0 length [m]',
       'Needle holder stitch 0 visibility [s]',
       'Needle holder stitch 0 velocity',
       'Needle holder stitch 0 velocity std',
       'Needle holder stitch 0 velocity above threshold',
       'Needle holder stitch 0 velocity threshold [m/s]',
       'Needle holder stitch 0 unit', 'Needle holder stitch 0 visibility [%]',
       'Forceps stitch 0 area presence [%]',
       'Forceps stitch 0 median area presence [%]',
       'Forceps stitch 0 length [m]', 'Forceps stitch 0 visibility [s]',
       'Forceps stitch 0 velocity', 'Forceps stitch 0 velocity std',
       'Forceps stitch 0 velocity above threshold',
       'Forceps stitch 0 velocity threshold [m/s]', 'Forceps stitch 0 unit',
       'Forceps stitch 0 visibility [%]',
       'Scissors stitch 0 area presence [%]',
   

## Select data

In [72]:
# remove all records with filename NA or NaN
dfs = df[df["filename"].notna()]
dfs = dfs.copy()
print(len(dfs))

# take all records with filename containing "Good", "Medium", "Bad", if the filename is the same, take the last one

# dfs = dfs[dfs["filename"].str.contains("Good|Medium|Bad")].drop_duplicates(subset=["filename"], keep="last")

# take the records uploaded after 2024-02-01
# uploaded_at to datetime
dfs["uploaded_at"] = pd.to_datetime(dfs["uploaded_at"])
dfs["timestamp"] = pd.to_datetime(dfs["timestamp"])
# dfs = dfs[dfs["uploaded_at"] > "2024-02-01"]
dfs = dfs[dfs["timestamp"] > "2024-07-03 8:19:59"]
# dfs = dfs[dfs["annotation_annotation_i"] == 0]
dfs = dfs[
    (dfs["annotation_annotation_i"] == 0) |
    dfs["collections"].str.contains("Ekn").fillna(False) 
]

# dfs = dfs[dfs["filename"].str.contains("Einzelknopfnaht")].drop_duplicates(subset=["filename"], keep="last")
# dfs = dfs[dfs["filename"].str.contains("|".join(filename_contains_list))].drop_duplicates(subset=["filename"], keep="last")
dfs = dfs.drop_duplicates(subset=["filename"], keep="last")
dfs = dfs[
    dfs["collections"].str.contains("dataset 2024-06").fillna(False) |
    dfs["collections"].str.contains("Ekn").fillna(False) 
]
# dfs = dfs[~dfs["collections"].str.contains("skin_model").fillna(False)] # remove artificial skin

print(len(dfs))
dfs["filename"]

298
59


158                                        Ekn_Ana_3.mp4
202                                        Ekn_Ana_2.mp4
203                                        Ekn_Ana_4.mp4
204                                        Ekn_Ana_1.mp4
205                                        Ekn_Ana_5.mp4
206                                       Ekn_Ana_10.mp4
207                                        Ekn_Ana_8.mp4
208                                        Ekn_Ana_9.mp4
209                                        Ekn_Ana_7.mp4
210                                       Ekn_Ana_13.mp4
211                                       Ekn_Ana_11.mp4
212                                       Ekn_Ana_14.mp4
213                                       Ekn_Ana_12.mp4
215                                        Ekn_Ana_6.mp4
216                                       Ekn_Ana_18.mp4
217                                       Ekn_Ana_16.mp4
218                                       Ekn_Ana_19.mp4
219                            

## Check annotations with not good tracking and remove them from the data

In [73]:
dfs

,finished_at,filename,report_url,duration_s_get_parameters_for_crop_rotate_rescale,orig fps,orig totalframecount,collections,orig frame_width,orig frame_height,ffmpeg_subprocess_params,...,Needle holder bbox knot 1 length [m],Needle holder bbox knot 1 visibility [s],Needle holder bbox knot 1 velocity,Needle holder bbox knot 1 velocity std,Needle holder bbox knot 1 velocity above threshold,Needle holder bbox knot 1 velocity threshold [m/s],Needle holder bbox knot 1 unit,Needle holder bbox knot 1 visibility [%],Forceps bbox knot 1 area presence [%],Forceps bbox knot 1 median area presence [%]
158,2024-08-29 14:20,Ekn_Ana_3.mp4,http://suran.kky.zcu.cz:8000//uploader/web_rep...,"48,3045342",30.0,1682.0,['Ekn'],1920.0,1080.0,ffmpeg -i /webapps/piglegsurgery/piglegsurgery...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
202,2024-09-02 14:54,Ekn_Ana_2.mp4,http://suran.kky.zcu.cz:8000//uploader/web_rep...,"8,708333969",30.0,1624.0,['Ekn'],1920.0,1080.0,ffmpeg -i /webapps/piglegsurgery/piglegsurgery...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
203,2024-09-02 14:55,Ekn_Ana_4.mp4,http://suran.kky.zcu.cz:8000//uploader/web_rep...,"101,4723032",30.0,1660.0,['Ekn'],1920.0,1080.0,ffmpeg -i /webapps/piglegsurgery/piglegsurgery...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
204,2024-09-02 14:55,Ekn_Ana_1.mp4,http://suran.kky.zcu.cz:8000//uploader/web_rep...,"81,15913153",30.0,1523.0,['Ekn'],1920.0,1080.0,ffmpeg -i /webapps/piglegsurgery/piglegsurgery...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
205,2024-09-02 14:55,Ekn_Ana_5.mp4,http://suran.kky.zcu.cz:8000//uploader/web_rep...,"34,25678253",30.0,1357.0,['Ekn'],1920.0,1080.0,ffmpeg -i /webapps/piglegsurgery/piglegsurgery...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
206,2024-09-02 14:57,Ekn_Ana_10.mp4,http://suran.kky.zcu.cz:8000//uploader/web_rep...,"9,14791584",30.0,1254.0,['Ekn'],1920.0,1080.0,ffmpeg -i /webapps/piglegsurgery/piglegsurgery...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
207,2024-09-02 14:58,Ekn_Ana_8.mp4,http://suran.kky.zcu.cz:8000//uploader/web_rep...,"8,179324389",30.0,1533.0,['Ekn'],1920.0,1080.0,ffmpeg -i /webapps/piglegsurgery/piglegsurgery...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
208,2024-09-02 14:58,Ekn_Ana_9.mp4,http://suran.kky.zcu.cz:8000//uploader/web_rep...,"8,845123529",30.0,1429.0,['Ekn'],1920.0,1080.0,ffmpeg -i /webapps/piglegsurgery/piglegsurgery...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
209,2024-09-02 14:59,Ekn_Ana_7.mp4,http://suran.kky.zcu.cz:8000//uploader/web_rep...,"61,59262753",30.0,1447.0,['Ekn'],1920.0,1080.0,ffmpeg -i /webapps/piglegsurgery/piglegsurgery...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
210,2024-09-02 15:02,Ekn_Ana_13.mp4,http://suran.kky.zcu.cz:8000//uploader/web_rep...,"8,282631874",30.0,1708.0,['Ekn'],1920.0,1080.0,ffmpeg -i /webapps/piglegsurgery/piglegsurgery...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [74]:
import seaborn as sns
import pigleg_evaluation_tools as pet

### Replace "," with "."

In [75]:
# find the columns where its values contain single "," and digits
dfs = pet.replace_comma_with_dot(dfs)

### Convert bool to the numeric


In [76]:
# get list of columns containing "annotation_annotation" in their name
cols = dfs.columns[dfs.columns.str.contains("annotation_annotation")]
dfs = pet.convert_bool_to_numeric(dfs, cols)




skipped: annotation_annotation_id
skipped: annotation_annotation_annotation
skipped: annotation_annotation_created_at
skipped: annotation_annotation_updated_at
skipped: annotation_annotation_annotator
skipped: annotation_annotation_stars
skipped: annotation_annotation_needle_grabbed_correctly
skipped: annotation_annotation_needle_holder_stabilized
skipped: annotation_annotation_needle_pierced_at_first_try
skipped: annotation_annotation_needle_pierced_at_right_angle
skipped: annotation_annotation_needle_rotated_correctly_on_opposite_side
skipped: annotation_annotation_needle_rotated_correctly_on_students_side
skipped: annotation_annotation_forceps_grabs_the_edge
skipped: annotation_annotation_three_knots_per_stitch
skipped: annotation_annotation_knots_are_done_right
skipped: annotation_annotation_threads_shortened_appropriately
skipped: annotation_annotation_stitch_to_wound_distance_is_correct
skipped: annotation_annotation_distance_between_stitches_is_correct
skipped: annotation_annota

In [77]:
# assert sorted(list(dfs["annotation_annotation_knots_are_done_right"].unique())) == [0.0, 1.0]

### If there is just one stitch, put ths information into stitch 1 measurements

In [78]:
# in pre-test post-test this should not be necessary

# If column containing " stitch 1" in its name has empty values, fill them with  the values from the column with the same name but withuou " stitch 1"
# for col in dfs.columns[dfs.columns.str.contains(" stitch 0")]:
#     dfs[col] = dfs[col].fillna(dfs[col.replace(" stitch 0", "")])

In [79]:
# dfs[["Needle holder stitch 1 length [m]", "Needle holder length [m]"]]


### Remove rows and columns with lot of NaN values

In [80]:
# get numeric columns from dfs
numeric_cols = dfs.columns[dfs.apply(lambda x: x.dtype is float).any()]


dfs[numeric_cols].isna().sum(axis=1)


/tmp/ipykernel_17302/3757271245.py:2: FutureWarning:

Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.



158    0.0
202    0.0
203    0.0
204    0.0
205    0.0
206    0.0
207    0.0
208    0.0
209    0.0
210    0.0
211    0.0
212    0.0
213    0.0
215    0.0
216    0.0
217    0.0
218    0.0
219    0.0
220    0.0
221    0.0
222    0.0
223    0.0
224    0.0
225    0.0
226    0.0
228    0.0
229    0.0
230    0.0
256    0.0
257    0.0
259    0.0
261    0.0
263    0.0
264    0.0
265    0.0
267    0.0
268    0.0
270    0.0
272    0.0
274    0.0
276    0.0
277    0.0
278    0.0
280    0.0
281    0.0
283    0.0
284    0.0
285    0.0
286    0.0
287    0.0
288    0.0
289    0.0
291    0.0
292    0.0
293    0.0
294    0.0
295    0.0
296    0.0
297    0.0
dtype: float64

# Find records with more than one stitch

In [81]:
# import numpy as np
# bc = np.bincount([1,5,5,5,1,2,1])
# print(type(bc))
# bc

In [82]:
import pigleg_evaluation_tools as pet

# display(dfs.columns[dfs.columns.str.contains(" stitch 2")])
# display(dfs.columns[dfs.columns.str.contains("Stitch 2")])
# show records with no NA values in "Stitch 3" column

pet.rows_with_missing_values_for_stitch(dfs, 3)



,finished_at,filename,report_url,duration_s_get_parameters_for_crop_rotate_rescale,orig fps,orig totalframecount,collections,orig frame_width,orig frame_height,ffmpeg_subprocess_params,...,Needle holder bbox knot 1 length [m],Needle holder bbox knot 1 visibility [s],Needle holder bbox knot 1 velocity,Needle holder bbox knot 1 velocity std,Needle holder bbox knot 1 velocity above threshold,Needle holder bbox knot 1 velocity threshold [m/s],Needle holder bbox knot 1 unit,Needle holder bbox knot 1 visibility [%],Forceps bbox knot 1 area presence [%],Forceps bbox knot 1 median area presence [%]
285,2024-09-17 13:47,Einzelknopfnaht_Schwein_Kerstin_Preuß.mp4,http://suran.kky.zcu.cz:8000//uploader/web_rep...,10.350072,25.0,7377.0,"['Einzel', 'dataset 2024-06']",1920.0,1012.0,ffmpeg -i /webapps/piglegsurgery/piglegsurgery...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
289,2024-09-17 15:08,Stresstest_Einzelknopfnaht.mp4,http://suran.kky.zcu.cz:8000//uploader/web_rep...,20.944759,29.0,12862.0,"['Einzelknopfnaht', 'Einzelknopfnaht 02-2024',...",2160.0,1440.0,ffmpeg -i /webapps/piglegsurgery/piglegsurgery...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
292,2024-09-17 15:22,Bildschirmaufnahme_2022-10-11_um_11.20.24.mov,http://suran.kky.zcu.cz:8000//uploader/web_rep...,15.961235,60.0,50913.0,['dataset 2024-06'],1874.0,1366.0,ffmpeg -i /webapps/piglegsurgery/piglegsurgery...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
295,2024-09-17 15:44,"Hilger,Anke-Posttest.mp4",http://suran.kky.zcu.cz:8000//uploader/web_rep...,21.928704,25.0,7651.0,"['pre-test post-test', 'dataset 2024-06', 'ski...",2560.0,1920.0,ffmpeg -i /webapps/piglegsurgery/piglegsurgery...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
296,2024-09-17 15:45,Schiesser_David_post_test.mp4,http://suran.kky.zcu.cz:8000//uploader/web_rep...,9.279840,25.0,7543.0,"['pre-test post-test', 'dataset 2024-06', 'ski...",1280.0,720.0,ffmpeg -i /webapps/piglegsurgery/piglegsurgery...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [83]:
# dfs[dfs["Stitch 3 duration [s]"].notna()]

# Separate individual stitches

1. Extract stitch evaluation from text annotation
2. Separate stitch columns into individual rows

In [84]:
# Extract evaluation of stitch movement from annotation column

# text_ann = dfone["annotation_annotation_annotation"].values[-1]
# lst = [line for line in sorted(text_ann.split("\r\n")) if "stitch_end" in line]
## lst = ['00:01:12 stitch_end 3* ', '00:02:14 stitch_end 4.5*', '00:03:09 stitch_end 5*', '00:04:13 stitch_end 3*']
# find number in front of *
# stitch_evaluation = [float(line.split("*",1)[0].split(" ")[-1]) for line in lst]

# for stitch_number in range(0, 4):
#     col_name = f"movement evaluation stitch {stitch_number}"
#     dfs[col_name] = np.nan
# for every line in dfs i want to add column evaluation stitch 1, stitch 2, stitch 3, stitch 4
for dfs_index, row in dfs.iterrows():
    name = row["filename"]
    try:
        text_ann = row["annotation_annotation_annotation"]
        lst = [line for line in sorted(text_ann.split("\r\n")) if "stitch_end" in line]
        stitch_evaluation = [float(line.split("*",1)[0].split(" ")[-1]) for line in lst]
        for stitch_number in range(0, 5):
            col_name = f"movement evaluation stitch {stitch_number}"
            if stitch_number < len(stitch_evaluation):
                dfs.loc[dfs_index, col_name] = stitch_evaluation[stitch_number]
            else:
                dfs.loc[dfs_index, col_name] = np.nan
    except Exception as e:
        print(f"Error in {name}: {e}")


Error in Ekn_Ana_13.mp4: 'float' object has no attribute 'split'
Error in Ekn_Ana_11.mp4: 'float' object has no attribute 'split'
Error in Ekn_Ana_14.mp4: 'float' object has no attribute 'split'
Error in Ekn_Ana_12.mp4: 'float' object has no attribute 'split'
Error in Ekn_Ana_18.mp4: 'float' object has no attribute 'split'
Error in Ekn_Ana_16.mp4: 'float' object has no attribute 'split'
Error in Ekn_Ana_19.mp4: 'float' object has no attribute 'split'
Error in Ekn_Ana_20.mp4: 'float' object has no attribute 'split'
Error in Ekn_Ana_17.mp4: 'float' object has no attribute 'split'
Error in Ekn_Ana_21.mp4: 'float' object has no attribute 'split'
Error in Ekn_Ana_23.mp4: 'float' object has no attribute 'split'
Error in Ekn_Ana_22.mp4: 'float' object has no attribute 'split'
Error in Ekn_CSW_2.mp4: 'float' object has no attribute 'split'
Error in Ekn_CSW_4.mp4: 'float' object has no attribute 'split'
Error in Ekn_CSW_1.mp4: 'float' object has no attribute 'split'
Error in Ekn_CSW_3.mp4: 'flo

In [85]:
# list(range(0, 4))

In [86]:
# I have columns named in pattern "Needle holder stitch 1 length [m]", "Needle holder stitch 2 length [m]", "Needle holder stitch 3 length [m]", "Needle holder stitch 4 length [m]". I would like to split each row into several rows, each containing just one stitch length and number of the stitch.
# searched_stitch_number = 4
keep_cols = ["filename", "annotation_annotation_annotation", "annotation_annotation_time_and_movements", "report_url"]
dfst = pet.new_dataframe_with_one_row_per_stitch(dfs, 5, keep_cols=keep_cols)
dfst = pet.convert_object_to_numeric(dfst)

Index(['Stitch 0 duration [s]', 'Stitch 0 start at [s]',
       'Needle holder stitch 0 area presence [%]',
       'Needle holder stitch 0 median area presence [%]',
       'Needle holder stitch 0 length [m]',
       'Needle holder stitch 0 visibility [s]',
       'Needle holder stitch 0 velocity',
       'Needle holder stitch 0 velocity std',
       'Needle holder stitch 0 velocity above threshold',
       'Needle holder stitch 0 velocity threshold [m/s]',
       'Needle holder stitch 0 unit', 'Needle holder stitch 0 visibility [%]',
       'Forceps stitch 0 area presence [%]',
       'Forceps stitch 0 median area presence [%]',
       'Forceps stitch 0 length [m]', 'Forceps stitch 0 visibility [s]',
       'Forceps stitch 0 velocity', 'Forceps stitch 0 velocity std',
       'Forceps stitch 0 velocity above threshold',
       'Forceps stitch 0 velocity threshold [m/s]', 'Forceps stitch 0 unit',
       'Forceps stitch 0 visibility [%]',
       'Scissors stitch 0 area presence [%]',
   

In [87]:
dfst.columns[dfst.columns.str.contains("movement")]

Index(['annotation_annotation_time_and_movements', 'movement evaluation stitch'], dtype='object')

# Filter out empty rows 

In [88]:

basic_column = ["Stitch duration [s]"]
basic_column = ["Needle holder stitch visibility [s]"]
# select just rows where column is notna

dfvalid = dfst.dropna(subset=basic_column)
dfvalid.reset_index(drop=True, inplace=True)
dfvalid = dfvalid.copy()
# dfst =dfall
# dfst[cols_X + [relevant_column]].notna().sum()
# dfst[selected_columns].notna().sum()
dfvalid.shape


(93, 172)

## Manually add information about data

In [89]:

# Adding some data
dfvalid["done_by_expert"] = dfvalid["filename"].str.contains("Ekn").astype(float)
# setting evaluation to 5 for expert
dfvalid.loc[dfvalid["done_by_expert"]==1., "movement evaluation stitch"] = 5
# dfvalid.loc[:, "done_by_expert"] = False
assert dfvalid["done_by_expert"].sum() > 0

dfst = dfvalid

# Save the output to the Excel file

In [90]:
!pwd

/webapps/piglegsurgery/devel


In [91]:
# datetime as string
import datetime
dt_str = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
fn = f"2024-07_list_of_all_stitches_sorted_by_duration.{dt_str}.xlsx"
dfst.to_excel(fn)

In [92]:


selected_cols = ['filename', 'stitch_id'] + list(dfst.columns[dfst.columns.str.contains("duration")])
selected_cols
dfst[selected_cols]



,filename,stitch_id,Stitch duration [s],Knot duration [s],Piercing duration [s]
0,Ekn_Ana_3.mp4,0,53.0,30.0,23.0
1,Ekn_Ana_2.mp4,0,50.0,31.0,19.0
2,Ekn_Ana_4.mp4,0,51.0,31.0,20.0
3,Ekn_Ana_1.mp4,0,48.0,24.0,24.0
4,Ekn_Ana_5.mp4,0,43.0,26.0,17.0
...,...,...,...,...,...
88,Schiesser_David_post_test.mp4,3,58.0,39.0,19.0
89,Einzelknopfnaht_Schwein_Kerstin_Preuß.mp4,4,50.0,24.0,26.0
90,Stresstest_Einzelknopfnaht.mp4,4,54.0,31.0,23.0
91,Bildschirmaufnahme_2022-10-11_um_11.20.24.mov,4,101.0,45.0,56.0


In [93]:
16//3

5

In [94]:
# print dtype of columns
dfst.dtypes

filename                                                 object
annotation_annotation_annotation                         object
annotation_annotation_time_and_movements                float64
report_url                                               object
Stitch duration [s]                                     float64
                                                         ...   
Needle holder bbox piercing velocity above threshold    float64
Needle holder bbox piercing velocity threshold [m/s]    float64
Needle holder bbox piercing unit                         object
Needle holder bbox piercing visibility [%]              float64
done_by_expert                                          float64
Length: 173, dtype: object

In [95]:
relevant_column = "movement evaluation stitch"

In [96]:

import plotly.express as px
px.scatter(dfst, y="Needle holder stitch length [m]", x=relevant_column, hover_data=["filename", "stitch_id"], width=500)
